In [44]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time

In [82]:
limit = 8 # setting to 3 articles limit per page

def getDate(url):
    try :
        req = Request(
            url=url, 
            headers={'User-Agent': 'Mozilla/5.0'}
        )
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, 'html.parser')
        dates = soup.find_all(class_ = "date") 
        dateArray=[]
        count = 1
        for date in dates :
            d = date.get_text().split()[1:]
            if d :
                dateArray.append(d)
            else :
                break
            if count >= limit :
                break
            count += 1    
        return (dateArray,count)
    except :
        print("error")


# tweak start and end for faster compile
def getStartEnd(url,startnum,endnum):
    try :
        check = True
        start,count = getDate("{}{}".format(url,startnum))
        while check :
            if (start[0][0] == 'Aug' and start[0][2] == '2022') :
                check = False
                break
            startnum += 1
            start,count = getDate("{}{}".format(url,startnum))
            print(start)
        print(startnum)
        # coronavirus special - oldest date to mar 2020
        if url == 'https://www.investing.com/news/coronavirus/' :
            endnum = 650 # last updated 17 October 2022
        else :
            check = True
            end,count = getDate("{}{}".format(url,endnum))
            while check :
                if end[0][0] == 'Jan' and end[0][2] == '2018' :
                    check = False
                    break
                endnum += 1
                end,count = getDate("{}{}".format(url,endnum))    
                print(end)
            print(endnum)
        return (startnum,endnum)
    except : 
        print("error")

def getArticles(url,startnum,endnum):
    df = pd.DataFrame(columns=['Date','Title'])
    try :
        start,end = getStartEnd(url,startnum,endnum)
        while start <= end :
            req = Request(
                url="{}{}".format(url,start), 
                headers={'User-Agent': 'Mozilla/5.0'}
            )
            webpage = urlopen(req).read() 
            soup = BeautifulSoup(webpage, 'html.parser')
            dates = soup.find_all(class_="date")
            titles = soup.find_all(class_ = "title") 
            for i in range(limit) :
                d = dates[i].get_text().split()[1:]
                if d :
                    df_new = pd.DataFrame({"Date" : dates[i].text[3:], "Title" : titles[i].text},index=[1])
                    df = pd.concat([df,df_new],ignore_index=True)
                else :
                    break
            print(start, end, "[Page {}/{}]".format(start, end))
            start += 1
            time.sleep(0.5)
        return df
    except :
        print("error")

In [57]:
df_tsla = getArticles("https://www.investing.com/equities/tesla-motors-news/",42,817)
df_tsla.to_csv('../Data/News/Stock/tsla_news.csv',index=False)

[['Sep', '02,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022']]
[['Aug', '31,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022']]
42
[['Feb', '05,', '2018'], ['Feb', '04,', '2018'], ['Feb', '02,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Jan', '31,', '2018'], ['Jan', '29,', '2018'], ['Jan', '29,', '2018']]
[['Jan', '28,', '2018'], ['Jan', '26,', '2018'], ['Jan', '25,', '2018'], ['Jan', '25,', '2018'], ['Jan', '25,', '2018'], ['Jan', '24,', '2018'], ['Jan', '24,', '2018'], ['Jan', '24,', '2018']]
817
error


AttributeError: 'NoneType' object has no attribute 'to_csv'

In [55]:
df_aapl = getArticles("https://www.investing.com/equities/apple-computer-inc-news/",30,1034) 
print(df_aapl.head())
print(df_aapl.shape)
df_aapl.to_csv('../Data/News/Stock/aapl_news.csv',index=False)

[['Sep', '06,', '2022'], ['Sep', '06,', '2022'], ['Sep', '05,', '2022'], ['Sep', '04,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022']]
[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022']]
[['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022']]
33
[['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018'], ['Feb', '02,', '2018']]
[['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018'], ['Feb', '01,', '2018']]
[['Feb', '01,', '2018'], ['Feb',

AttributeError: 'NoneType' object has no attribute 'head'

In [50]:
df_meta = getArticles("https://www.investing.com/equities/facebook-inc-news/",20,1211) 
print(df_meta.head())
print(df_meta.shape)
df_meta.to_csv('../Data/News/Stock/meta_news.csv',index=False)

[['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '02,', '2022'], ['Sep', '01,', '2022'], ['Sep', '01,', '2022'], ['Aug', '31,', '2022'], ['Aug', '31,', '2022']]
[['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '29,', '2022'], ['Aug', '29,', '2022']]
22
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
1212
22 1212 [Page 22/1212]
23 1212 [Page 23/1212]
24 1212 [Page 24/1212]
25 1212 [Page 25/1212]
26 1212 [Page 26/1212]
27 1212 [Page 27/1212]
28 1212 [Page 28/1212]
29 1212 [Page 29/1212]
30 1212 [Page 30/1212]
31 1212 [Page 31/1212]
32 1212 [Page 32/1212]
33 1212 [Page 33/1212]
34 1212 [Page 34/1212]
35 1212 [Page 35/1212]
36 1212 [Page 36/1212]
37 1212 [Page 37/1212]
38 1212 [Page 38/1212]
39 1212 [Page 39/1212]

In [83]:
df_world = getArticles("https://www.investing.com/news/world-news/",105,4320)
print(df_world.head())
print(df_world.shape)
df_world.to_csv('../Data/News/Global/world_news.csv',index=False)

105
4320
105 4320 [Page 105/4320]
106 4320 [Page 106/4320]
107 4320 [Page 107/4320]
108 4320 [Page 108/4320]
109 4320 [Page 109/4320]
110 4320 [Page 110/4320]
111 4320 [Page 111/4320]
112 4320 [Page 112/4320]
113 4320 [Page 113/4320]
114 4320 [Page 114/4320]
115 4320 [Page 115/4320]
116 4320 [Page 116/4320]
117 4320 [Page 117/4320]
118 4320 [Page 118/4320]
119 4320 [Page 119/4320]
120 4320 [Page 120/4320]
121 4320 [Page 121/4320]
122 4320 [Page 122/4320]
123 4320 [Page 123/4320]
124 4320 [Page 124/4320]
125 4320 [Page 125/4320]
126 4320 [Page 126/4320]
127 4320 [Page 127/4320]
128 4320 [Page 128/4320]
129 4320 [Page 129/4320]
130 4320 [Page 130/4320]
131 4320 [Page 131/4320]
132 4320 [Page 132/4320]
133 4320 [Page 133/4320]
134 4320 [Page 134/4320]
135 4320 [Page 135/4320]
136 4320 [Page 136/4320]
137 4320 [Page 137/4320]
138 4320 [Page 138/4320]
139 4320 [Page 139/4320]
140 4320 [Page 140/4320]
141 4320 [Page 141/4320]
142 4320 [Page 142/4320]
143 4320 [Page 143/4320]
144 4320 [Page 1

In [84]:
df_politics = getArticles("https://www.investing.com/news/politics/",9,780) 
print(df_politics.head())
print(df_politics.shape)
df_politics.to_csv('../Data/News/Global/politics_news.csv',index=False)

[['Aug', '31,', '2022'], ['Aug', '31,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022'], ['Aug', '30,', '2022']]
10
[['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018'], ['Jan', '31,', '2018']]
781
10 781 [Page 10/781]
11 781 [Page 11/781]
12 781 [Page 12/781]
13 781 [Page 13/781]
14 781 [Page 14/781]
15 781 [Page 15/781]
16 781 [Page 16/781]
17 781 [Page 17/781]
18 781 [Page 18/781]
19 781 [Page 19/781]
20 781 [Page 20/781]
21 781 [Page 21/781]
22 781 [Page 22/781]
23 781 [Page 23/781]
24 781 [Page 24/781]
25 781 [Page 25/781]
26 781 [Page 26/781]
27 781 [Page 27/781]
28 781 [Page 28/781]
29 781 [Page 29/781]
30 781 [Page 30/781]
31 781 [Page 31/781]
32 781 [Page 32/781]
33 781 [Page 33/781]
34 781 [Page 34/781]
35 781 [Page 35/781]
36 781 [Page 36/781]
37 781 [Page 37/781]
38 781 [Page 38/781

In [85]:
df_coronavirus = getArticles("https://www.investing.com/news/coronavirus/",3,650) 
print(df_coronavirus.head())
print(df_coronavirus.shape)
df_coronavirus.to_csv('../Data/News/Global/coronavirus_news.csv',index=False)

3
3 650 [Page 3/650]
4 650 [Page 4/650]
5 650 [Page 5/650]
6 650 [Page 6/650]
7 650 [Page 7/650]
8 650 [Page 8/650]
9 650 [Page 9/650]
10 650 [Page 10/650]
11 650 [Page 11/650]
12 650 [Page 12/650]
13 650 [Page 13/650]
14 650 [Page 14/650]
15 650 [Page 15/650]
16 650 [Page 16/650]
17 650 [Page 17/650]
18 650 [Page 18/650]
19 650 [Page 19/650]
20 650 [Page 20/650]
21 650 [Page 21/650]
22 650 [Page 22/650]
23 650 [Page 23/650]
24 650 [Page 24/650]
25 650 [Page 25/650]
26 650 [Page 26/650]
27 650 [Page 27/650]
28 650 [Page 28/650]
29 650 [Page 29/650]
30 650 [Page 30/650]
31 650 [Page 31/650]
32 650 [Page 32/650]
33 650 [Page 33/650]
34 650 [Page 34/650]
35 650 [Page 35/650]
36 650 [Page 36/650]
37 650 [Page 37/650]
38 650 [Page 38/650]
39 650 [Page 39/650]
40 650 [Page 40/650]
41 650 [Page 41/650]
42 650 [Page 42/650]
43 650 [Page 43/650]
44 650 [Page 44/650]
45 650 [Page 45/650]
46 650 [Page 46/650]
47 650 [Page 47/650]
48 650 [Page 48/650]
49 650 [Page 49/650]
50 650 [Page 50/650]
51 6